In [1]:
%matplotlib tk
import matplotlib.pyplot as plt
import numpy as np
from utils.config_jupyter_notebooks import *
from utils.generative_models import *

L :  1.99951171875  vs length of windows  2  source: config_jupyter_notebooks  vs distances of time domain strip:  (0.00024411081903918935,)  which should equal  0.00024411081903918935
Important variables: 
		signal_strip_time, signal_strip_strain 
		signal_strip_strain_tapered
		strain
		time_domain_strip
		N


L :  1.99951171875  vs length of windows  2  source: config_jupyter_notebooks  vs distances of time domain strip:  (0.00024411081903918935,)  which should equal  0.00024411081903918935
Important variables: 
		signal_strip_time, signal_strip_strain 
		signal_strip_strain_tapered
		strain
		time_domain_strip
		N




In [2]:
def divisor(t: np.array, x: np.array, window_length: float):
    """
    Subdivides a given (x, t) time series into non-overlapping windows of duration `window_length` (in seconds).
    Assumes t is sorted and uniformly sampled.

    :param x:               The time series values.
    :param t:               The time stamps (1D array, same length as x).
    :param window_length:   Window length in seconds.
    :return:                Tuple of (windows of t, windows of x), both shaped (num_windows, samples_per_window)
    """
    dt = t[1] - t[0]  # assume uniform sampling
    samples_per_window = int(window_length / dt)

    total_samples = len(t)
    usable_samples = (total_samples // samples_per_window) * samples_per_window

    x = x[:usable_samples]
    t = t[:usable_samples]

    res_x = x.reshape(-1, samples_per_window)
    res_t = t.reshape(-1, samples_per_window)

    return res_t, res_x



def divisor_unit_test():
    x = np.linspace(1, 8, 6)
    t = np.linspace(2, 10, 6)  # dt = 1.6, so 2 samples = 3.2s
    wl = 3.2  # seconds → 2 samples

    res_t, res_x = divisor(x, t, wl)

    expected_t = np.array([[2., 3.6], [5.2, 6.8], [8.4, 10.]])
    expected_x = np.array([[1., 2.4], [3.8, 5.2], [6.6, 8.]])

    assert np.allclose(res_t, expected_t), "Time values do not match"
    assert np.allclose(res_x, expected_x), "Data values do not match"
    print("Test passed!")



In [111]:

def xcorr_data(d1, d2, maxlags=None, normed=True):
    # IS united tested: returns the same thing as plt.xcorr!
    d1 = np.asarray(d1)
    d2 = np.asarray(d2)
    N = len(d1)

    if maxlags is None:
        maxlags = N - 1

    lags = np.arange(-maxlags, maxlags + 1)
    c = np.correlate(d1 - np.mean(d1), d2 - np.mean(d2), mode='full')

    if normed:
        scale = np.std(d1) * np.std(d2) * N
        c = c / scale

    mid = len(c) // 2
    start = mid - maxlags
    stop = mid + maxlags + 1
    return lags, c[start:stop]

def cross_correlate(d1, d2):
    """finds the cross-correlation between two arrays d1 and d2"""
    # res = plt.xcorr(d1, d2, maxlags=len(d1) - 1)
    res = xcorr_data(d1, d2, maxlags=len(d1) - 1)
    return res

def get_correlation(H1_data, L1_data, time_arr, t_min, t_max):
    """obtains the cross correlation between the H1 and L1 strain data arrays,
    limited to times in between t_min and t_max of the corresponding time array.

    Args:
        H1_data (ndarray): H1 strain data
        L1_data (ndarray): L1 strain data
        time_arr (ndarray): Time corresponding to both H1 and L1 data.
        t_min (float): Minimum time for data cross-correlation
        t_max (float): Maximum time for data cross-correlation

    Returns:
        ndarray: cross-correlation values obtained
        ndarray: time shifts for the given correlation values
    """
    # we're only doing this between t_min and t_max
    time_int = np.where((time_arr >= t_min) & (time_arr <= t_max))
    H1_int = H1_data[time_int]
    L1_int = L1_data[time_int]

    correlation = cross_correlate(H1_int, L1_int)
    plt.close()
    lags = correlation[0]
    corr = correlation[1]
    # here we have one correlation value for each possible lag, total amount is
    # 2 * len(t_int)
    interval_len = (t_max - t_min)
    time_diffs = np.linspace(-1 * interval_len, interval_len, len(corr))
    return corr, time_diffs


def run_cross_corrs(orig_H1, orig_L1, orig_time, t0, tmin=.25, tmax=.46, plot_datasets_themselves=True):
    """Runs 4 different cross correlations for the H1, L1 residual data and time,
    the H1, L1 original strain data and time: Full interval, first half, last
    half, and .39-43s.

    Args:
        orig_H1 (ndarray): original H1 strain data from event
        orig_L1 (ndarray): original L1 strain data from event
        orig_time (ndarray): time corresponding to orig data
        t0 (float): starting time of time interval
        tmin (float): starting time of full correlation interval (i.e. which min time at which to start the correlation)
        tmax (float): ending time of full correlation interval (max time to start the correlation)
        plot_datasets_themselves: Whether to plot datasets themselves or not.

    Returns:
        dict: dictionary containing residual correlation values, whose keys
            are the various time intervals
        dict: dictionary containing original strain correlation values, whose
            keys are the various time intervals
    """
    # define 4 different intervals for cross correlation
    intervals = {f'corr': (t0 + tmin, t0 + tmax)}
    # intervals = {'All 0.2s': (t0 + s, t0 + e), 'Last half': (t0 + s + .1, t0 + e),
    #             'First half': (t0 + s, t0 + s + .1), '0.39s-0.43s': (t0 + .39, t0 + .43)}

    resid_correlation_vals = {}
    original_correlation_vals = {}
    # go through and calculate for each interval
    for key in intervals.keys():
        interval = intervals[key]
        detrend_orig_H1 = orig_H1 - np.mean(orig_H1)
        detrend_orig_L1 = orig_L1 - np.mean(orig_L1)
        orig_corr = get_correlation(detrend_orig_H1, detrend_orig_L1, orig_time, interval[0],
                                    interval[1])

        if plot_datasets_themselves:
            # plot the datasets too
            plt.plot(orig_time, detrend_orig_H1, label="Original dataset1 detrended")
            plt.plot(orig_time, detrend_orig_L1, label="Original dataset2 detrended")
            plt.legend()
            plt.show()

        original_correlation_vals[key] = orig_corr

    return original_correlation_vals

In [107]:
def plot_correlation_vals_axis(correlation_vals, ax):
    """Plots correlations values obtained on a given axis object."""
    colors = {'corr': 'blue'}
    # colors = {'First half': 'black', 'Last half': 'red', 'All 0.2s': 'blue',
    #           '0.39s-0.43s': 'green'}
    if ax is None:
        fig, ax = plt.subplots()
    for key in correlation_vals.keys():
        corr = correlation_vals[key]
        # plot in ms instead of s
        ax.plot(corr[1] * 1000, corr[0], colors[key], label=key)

    yVals = np.linspace(-1, 1, 1000)
    ax.plot([7] * 1000, yVals, '--', color='gray', linewidth=.5)  # just for the 7-10 millisecond time delay is that would be the expected difference between the interferometers seen with wh and bp data
    ax.set_ylabel(r'C($\tau$)')
    ax.set_xlabel(r'$\tau$ (ms)')
    ax.legend(loc='upper left')

def single_corr_plot(corr_val, title):
        plot_correlation_vals_axis(corr_val, None)
        plt.title(title)
        plt.vlines(45, -1, 1, color="red")
        plt.vlines(-20, -1, 1, color="green")
        plt.show()

In [45]:
strain_object_L1 = unpickle_me_this("data/GW150914_strain_L1.pickle")
time_H1 = time
strain_H1 = strain.value * 1e19
time_L1 = np.array(strain_object_L1.times) - zero_time
strain_L1 = strain_object_L1.value * 1e19

In [108]:
idcs_of_event = np.where((time_H1>16.) & (time_H1<16.5))
strain_L1_in_significant_interval = strain_L1[idcs_of_event]
strain_H1_in_significant_interval = strain_H1[idcs_of_event]
time_in_significant_interval = time[idcs_of_event]

def another_wrapper(dataset1_in_window, dataset2_in_window, time_in_window, plot_datasets_themselves=False, show_plot=False):

    myCorrelationVals = run_cross_corrs(dataset1_in_window, dataset2_in_window, time_in_window, t0=0, tmin=np.min(time_in_window), tmax=np.max(time_in_window), plot_datasets_themselves=plot_datasets_themselves)
    tmp = myCorrelationVals["corr"]
    corr_values = tmp[0]
    time_delay_values = tmp[1]*1000  # in ms
    max_idx = np.argmax(corr_values)
    min_idx = np.argmin(corr_values)
    cmax = corr_values[max_idx]
    cmin = corr_values[min_idx]
    sigma = np.std(corr_values)

    cmax_sigma = np.round(cmax/sigma,2)
    cmin_sigma = np.round(cmin/sigma,2)

    print(f"[{np.round(np.min(time_in_window),2)} -> {np.round(np.max(time_in_window),2)}]s : " , "corr_max, corr_min: \t", cmax, f"({cmax_sigma}σ)","\t", cmin, f"({cmin_sigma}σ)", "\t at time delays ", time_delay_values[max_idx],"\t", time_delay_values[min_idx])

    if show_plot:
        single_corr_plot(myCorrelationVals, " my corr")

    return cmax, cmin, cmax_sigma, cmin_sigma, time_delay_values[max_idx], time_delay_values[min_idx]

print("length of time domain: ", len(time_in_significant_interval))
print("length of correlation: ")
# example usage:
another_wrapper(strain_H1_in_significant_interval, strain_L1_in_significant_interval, time_in_significant_interval, plot_datasets_themselves=True, show_plot=True)

length of time domain:  2047
length of correlation: 
[16.0 -> 16.5]s :  corr_max, corr_min: 	 5044.518823703457 (2.89σ) 	 -4998.447780105997 (-2.87σ) 	 at time delays  45.166015625 	 -21.728515625


(5044.518823703457,
 -4998.447780105997,
 2.89,
 -2.87,
 45.166015625,
 -21.728515625)

In [125]:
wl = 0.5

div_object_l1 = divisor(time_L1, strain_L1, window_length=wl)
div_object_h1 = divisor(time_H1, strain_H1, window_length=wl)

time_windows_l1 = div_object_l1[0]
strain_windows_l1 = div_object_l1[1]

time_windows_h1 = div_object_h1[0]
strain_windows_h1 = div_object_h1[1]

res = []
res_sigma = []
for i in range(len(time_windows_l1)):
    cmax, _, cmax_sigma, _, _, _ = another_wrapper(strain_windows_h1[i], strain_windows_l1[i], time_windows_h1[i], plot_datasets_themselves=False, show_plot=False)
    res.append(cmax)
    res_sigma.append(cmax_sigma)

print("\n\n")
largest_cmax_from_sigma_index = np.argmax(res_sigma)
print("largest cmax in sigmas ", res[largest_cmax_from_sigma_index], " in the interval ", time_windows_h1[largest_cmax_from_sigma_index], "which was ", res_sigma[largest_cmax_from_sigma_index], " σ")

res = np.array(res)
all_above_treshhold = np.where(res > 0.75)
print("\n\nor, all that are absolutely above 0.75: ")
for i in all_above_treshhold[0]:
    print("maximum correlation of ", res[i], " in window ", np.min(time_windows_h1[i]), " -> ", np.max(time_windows_h1[i]))

[0.0 -> 0.5]s :  corr_max, corr_min: 	 0.738824845437958 (2.5σ) 	 -0.6719654406344554 (-2.28σ) 	 at time delays  101.806640625 	 174.8046875
[0.5 -> 1.0]s :  corr_max, corr_min: 	 0.7636549946393925 (2.32σ) 	 -0.7625994415544737 (-2.32σ) 	 at time delays  -65.185546875 	 18.5546875
[1.0 -> 1.5]s :  corr_max, corr_min: 	 0.5854399006163533 (2.59σ) 	 -0.6048776616188857 (-2.68σ) 	 at time delays  -33.447265625 	 31.494140625
[1.5 -> 2.0]s :  corr_max, corr_min: 	 0.4942879781938037 (2.13σ) 	 -0.5123783944132522 (-2.21σ) 	 at time delays  9.765625 	 81.787109375
[2.0 -> 2.5]s :  corr_max, corr_min: 	 0.6848620716989723 (2.69σ) 	 -0.673006340196611 (-2.65σ) 	 at time delays  211.42578125 	 143.798828125
[2.5 -> 3.0]s :  corr_max, corr_min: 	 0.6498386571877365 (2.62σ) 	 -0.6574718146798525 (-2.65σ) 	 at time delays  -215.8203125 	 -279.052734375
[3.0 -> 3.5]s :  corr_max, corr_min: 	 0.42655767812711803 (2.48σ) 	 -0.3916573568572242 (-2.28σ) 	 at time delays  407.958984375 	 -41.50390625
[